In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.vectorstores import InMemoryVectorStore
from agents import OpenAIChatCompletionsModel
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import Tool
from dotenv import load_dotenv
from openai import AsyncOpenAI
from langchain.agents import initialize_agent
import os
load_dotenv(override=True)

USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:
client = AsyncOpenAI(
    api_key=os.getenv("GEMINI_ACCESS_KEY"),
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)
model = OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=client
)

In [3]:
# Gemini for reasoning/answer generation
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-001",google_api_key=os.getenv('GEMINI_ACCESS_KEY'))  # or "gemini-1.5-pro"

# Gemini embeddings for vector store
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=os.getenv('GEMINI_ACCESS_KEY'))

In [5]:
langGraph_urls = [
    "https://langchain-ai.github.io/langgraph",
    "https://academy.langchain.com/courses/intro-to-langgraph",
    "https://langchain-ai.github.io/langgraph/concepts/why-langgraph",
    "https://langchain-ai.github.io/langgraph/tutorials/rag/langgraph_agentic_rag",
    "https://www.datacamp.com/tutorial/langgraph-tutorial",
    "https://medium.com/%40wendell_89912/building-an-agentic-rag-with-langgraph-a-step-by-step-guide-009c5f0cce0a",
    "https://huggingface.co/learn/agents-course/en/unit2/langgraph/when_to_use_langgraph"
]

langChain_urls=[
   "https://python.langchain.com/docs/introduction/",
    "https://docs.langchain.com/oss/python/overview",
    "https://www.langchain.com",
    "https://python.langchain.com/api_reference",
    "https://academy.langchain.com/",
    "https://www.reddit.com/r/LangChain/"
]

In [11]:
# Load & split
# docs = [WebBaseLoader(url).load()[0] for url in urls]
docs = []
for url in langGraph_urls:
    loaded = WebBaseLoader(url).load()
    docs.extend(loaded) 

docs_list = docs
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=50)
# splits = splitter.split_documents(docs_list)
splits = splitter.split_documents(docs)

# Extract only text from the documents
texts = [doc.page_content for doc in splits]

# Now embed only the text
vectorstore = InMemoryVectorStore.from_texts(texts, embedding=embeddings)
retriever = vectorstore.as_retriever()

def retrieve_langgraph_docs(query: str) -> str:
    """Retrieve relevant docs from the vectorstore"""
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in results])

langGraph_retriever_tool = Tool(
    name="retrieve",
    func=retrieve_langgraph_docs,
    description="Retrieve relevant documents given a user query"
)


In [12]:
# Load & split
# docs = [WebBaseLoader(url).load()[0] for url in urls]
docs = []
for url in langChain_urls:
    loaded = WebBaseLoader(url).load()
    docs.extend(loaded) 

docs_list = docs
splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(chunk_size=1000, chunk_overlap=50)
# splits = splitter.split_documents(docs_list)
splits = splitter.split_documents(docs)

# Extract only text from the documents
texts = [doc.page_content for doc in splits]

# Now embed only the text
vectorstore = InMemoryVectorStore.from_texts(texts, embedding=embeddings)
retriever = vectorstore.as_retriever()

def retrieve_langchain_docs(query: str) -> str:
    """Retrieve relevant docs from the vectorstore"""
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([doc.page_content for doc in results])

langChain_retriever_tool = Tool(
    name="retrieve",
    func=retrieve_langchain_docs,
    description="Retrieve relevant documents given a user query"
)


In [13]:
tools = [langGraph_retriever_tool, langChain_retriever_tool]

In [14]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="openai-functions",
    verbose=True
)

result = agent.run("What is langChain")
print("Agentic RAG Answer:", result)




> Entering new AgentExecutor chain...
I need to find out what LangChain is. I should use the retrieve tool to search for information about it.
Action: retrieve
Action Input: "What is LangChain?"
Observation: Introduction | ü¶úÔ∏èüîó LangChain

LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Reddit - The heart of the internet






































Skip to main content



























Open menu

Open navigation



































































 
Go to Reddit Home




















r/LangChain

A chip








A close button













Log In

Log in to Reddit










Expand user menu
Open settings menu


 


 


 














































                  r/LangChain
                  
                



   members
                  




                                online
        

In [15]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="openai-functions",
    verbose=True
)

result = agent.run("What is langGraph?")
print("Agentic RAG Answer:", result)




> Entering new AgentExecutor chain...
I need to find out what LangGraph is. I can use the retrieve tool to search for information about it.
Action: retrieve
Action Input: "What is LangGraph?"
Observation: Introduction | ü¶úÔ∏èüîó LangChain

LangChain is a framework for developing applications powered by large language models (LLMs).
LangChain simplifies every stage of the LLM application lifecycle:

Moderator list hidden.

                Learn More
              



View all moderators









Reddit Rules



Privacy Policy



User Agreement



Accessibility


Reddit, Inc. © 2025. All rights reserved.








Expand Navigation








Collapse Navigation

Reddit - The heart of the internet






































Skip to main content



























Open menu

Open navigation



































































 
Go to Reddit Home




















r/LangChain

A chip








A close button













Log In

Log in to R